# Обучение без учителя: метод главных компонент и кластеризация

Датасет https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering, KMeans, SpectralClustering
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [4]:
X_train = np.loadtxt("samsung_HAR/samsung_train.txt")
y_train = np.loadtxt("samsung_HAR/samsung_train_labels.txt").astype(int)

X_test = np.loadtxt("samsung_HAR/samsung_test.txt")
y_test = np.loadtxt("samsung_HAR/samsung_test_labels.txt").astype(int)

In [5]:
X_train

array([[ 0.28858451, -0.02029417, -0.13290514, ..., -0.84124676,
         0.17994061, -0.05862692],
       [ 0.27841883, -0.01641057, -0.12352019, ..., -0.8447876 ,
         0.18028889, -0.05431672],
       [ 0.27965306, -0.01946716, -0.11346169, ..., -0.84893347,
         0.18063731, -0.04911782],
       ...,
       [ 0.27338737, -0.01701062, -0.04502183, ..., -0.77913261,
         0.24914484,  0.04081119],
       [ 0.28965416, -0.01884304, -0.15828059, ..., -0.78518142,
         0.24643223,  0.02533948],
       [ 0.35150347, -0.01242312, -0.20386717, ..., -0.78326693,
         0.24680852,  0.03669484]])

In [6]:
assert(X_train.shape == (7352, 561) and y_train.shape == (7352,))
assert(X_test.shape == (2947, 561) and y_test.shape == (2947,))

In [13]:
X_test.shape, X_train.shape

((2947, 561), (7352, 561))

In [16]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [17]:
X.shape, y.shape

((10299, 561), (10299,))

Определим число уникальных значений меток целевого класса.

In [18]:
np.unique(y)

array([1, 2, 3, 4, 5, 6])

In [20]:
n_classes = np.unique(y).size
n_classes

6

Эти метки соответствуют:

1 - ходьбе

2 - подъему вверх по лестнице

3 - спуску по лестнице

4 - сидению

5 - стоянию

6 - лежанию

Отмасштабируйте выборку с помощью StandardScaler с параметрами по умолчанию.

In [22]:
scaler = StandardScaler()
X_scaled =scaler.fit_transform(X)

Понижаем размерность с помощью PCA, оставляя столько компонент, сколько нужно для того, чтобы объяснить как минимум 90% дисперсии исходных (отмасштабированных) данных. Используйте отмасштабированную выборку и зафиксируйте random_state (константа RANDOM_STATE).

In [25]:
RANDOM_STATE = 17

pca = PCA(random_state=RANDOM_STATE)
X_pca = pca.fit_transform(X_scaled)



n_components = 0
explained_ratio = 0
while explained_ratio < 0.9:
    explained_ratio += pca.explained_variance_ratio_[n_components]
    n_components += 1
n_components

65

Сколько процентов дисперсии приходится на первую главную компоненту? Округлите до целых процентов

In [26]:
pca.explained_variance_ratio_[0]

0.5073822103501332